# `scan` - Looping in Theano

Following:

http://deeplearning.net/software/theano/library/scan.html#lib-scan

## Simple loop with accumulation

Suppose we want to compute `A**k` elementwise.

In [4]:
import theano
import theano.tensor as T

In [1]:
k = T.iscalar('k')
A = T.vector('A')

# symbolic description of the result
# ------------------------------------
# **note:** the order of the parameters in the lambda are fixed by scan - 
# the output of the prior call call to `fn` (or the initial value the first
# time through) is the first parameter, followed by all non-sequences
# **note:** the outputs are _initialized_ as a tensor with the same dtype
# and shape as `A`
# **note:** `A` is given to scan as a _non-sequence_ parameter, and we also
# specify the number of steps as a scalar
#
# `scan` returns a tuple containing the result and a dictionary of updates.
# in this case the dictionary is empty. the result is not a matrix, but a 
# 3D tensor containing A**k at each step - we want only the last one, so 
# will compile a function to get only that
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                             outputs_info=T.ones_like(A),
                             non_sequences=A,
                             n_steps=k)

# we only want A**k, but scan gives us A**1 ... A**k
# discard values we don't care about - 
# scan will notice this and won't waste memory saving them
# (this is a deliberate optimization)
final_result = result[-1]

# compile fn for A**k
power = theano.function(inputs=[A, k], outputs=final_result, updates=updates)

/Users/perdue/anaconda/envs/python2/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [2]:
print power(range(10), 2)

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]


In [3]:
print power(range(10), 4)

[  0.00000000e+00   1.00000000e+00   1.60000000e+01   8.10000000e+01
   2.56000000e+02   6.25000000e+02   1.29600000e+03   2.40100000e+03
   4.09600000e+03   6.56100000e+03]


## Iterating over the first dimension of a tensor: Calculating a polynomial

In addition to looping a fixed number of times, `scan` can iterate over the leading dimension of tensors (this is similar to `for x in list`). We must use the `sequence` keyword arg.

In [5]:
import numpy

In [46]:
coefficients = T.vector('coefficients')
x = T.scalar('x')

max_coefficients_supported = 10000

# generate the components of the polynomial
# we supply two sequences - the coefficient in front of the term and the power of each term
# (coming from the `arange`); the free variable is the non_sequenced term
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                outputs_info=None,
                                sequences=[coefficients, T.arange(max_coefficients_supported)],
                                non_sequences=x)

# sum them all up
# we could also run an accumulator here, which would be more memory efficient, but this is meant
# to be an illustrative example
polynomial = components.sum()

# compile the fn
calculate_polynomial = theano.function(inputs=[coefficients, x],
                                      outputs=polynomial)

In [47]:
test_coefficients = numpy.asarray([1, 0, 2], dtype=numpy.float32)
test_value = 3

In [48]:
print calculate_polynomial(test_coefficients, test_value)

19.0


In [49]:
print 1.0 * (3**0) + 0.0 * (3**1) + 2.0 * (3**2)

19.0


In [69]:
# try the accumulator
coefficients = T.vector('coefficients')
x = T.scalar('x')

# outputs_info=T.zeros_like(x),
# outputs_info=T.as_tensor_variable(numpy.asarray(0, x.dtype)),
comps, updates = theano.scan(fn=lambda coefficient, free_variable: 
                            coefficient * free_variable,
                            outputs_info=None,
                            sequences=[coefficients],
                            non_sequences=x)

polynomial_sum = comps

calc_polynomial_sum = theano.function(inputs=[coefficients, x],
                                     outputs=polynomial_sum)

In [71]:
test_coefficients = numpy.asarray([1, 2, 3, 4, 5], dtype=numpy.float32)
test_value = 1
print calc_polynomial_sum(test_coefficients, test_value)

[ 1.  2.  3.  4.  5.]


In [78]:
# try the accumulator
coefficients = T.vector('coefficients')
x = T.scalar('x')
k = T.iscalar('k')


# outputs_info=T.zeros_like(x),
# outputs_info=T.as_tensor_variable(numpy.asarray(0, x.dtype)),
comps, updates = theano.scan(fn=lambda prior_result, free_variable: 
                            prior_result + free_variable,
                            outputs_info=T.as_tensor_variable(numpy.asarray(0, x.dtype)),
                            non_sequences=x,
                            n_steps=k)

my_sum = comps

calc_my_sum = theano.function(inputs=[x, k],
                              outputs=my_sum)

In [79]:
test_value = 2
test_k = 4
print calc_my_sum(test_value, test_k)

[ 2.  4.  6.  8.]
